In [1]:
!pip install clickhouse_connect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.2/966.2 kB 15.9 MB/s eta 0:00:00a 0:00:01


In [2]:
!pip install fast-autocomplete[levenshtein] 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 58.2 MB/s eta 0:00:00:00:01


In [3]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.6 MB/s eta 0:00:0000:01


In [38]:
from transformers import AutoTokenizer, AutoModel
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import clickhouse_connect
from fast_autocomplete.misc import read_csv_gen
from fast_autocomplete import AutoComplete
from sentence_transformers import util
from sklearn.cluster import AgglomerativeClustering

In [5]:
class Text2MiniLM:
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = AutoModel.from_pretrained('intfloat/multilingual-e5-small').to(self.device)
        self.tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-small')
        
    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    
    def get_lm_embedding(self, text):
        encoded_input = self.tokenizer([text], padding=True, truncation=True, return_tensors='pt').to(self.device)

        with torch.no_grad():
            model_output = self.model(**encoded_input)

        sentence_embeddings = self.mean_pooling(model_output, encoded_input['attention_mask'])
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        
        return sentence_embeddings

In [7]:
client = clickhouse_connect.get_client(host='91.224.86.248', 
                                       port=8123, 
                                       username=..., 
                                       password=...)

TABLENAME = 'dictionary'

In [44]:
class AutocompleteService:
    def __init__(self, client):
        self.model = Text2MiniLM()
        self.client = client
        self.phrases_dict = self.get_dict()
        self.autocomplete = AutoComplete(
            words=self.phrases_dict,
            valid_chars_for_string='абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ'
        )

    def get_n_candidates(self, user_phrase):
        return self.autocomplete.search(word=user_phrase, max_cost=5, size=6)

    def get_dict(self):
        result = self.client.query('SELECT word FROM dictionary')
        dict_words = [row[0] for row in result.result_rows]
        return {phrase: {} for phrase in dict_words}

    def clastering_req(self, request):
        candidates_words = self.get_n_candidates(request)
        if not candidates_words:
            return [], []
        
        list_candidates = [word for sublist in candidates_words for word in sublist]
        list_embeddings = [self.model.get_lm_embedding(word).cpu().numpy().tolist() for word in list_candidates]
        list_embeddings = np.array(list_embeddings).reshape(len(list_candidates), -1).tolist()

        if len(list_candidates) == 1:
            return list_embeddings, list_candidates

        clustering = AgglomerativeClustering(
            n_clusters=None, distance_threshold=0.2, metric='cosine', linkage='average'
        ).fit(list_embeddings)
        labels = clustering.fit_predict(list_embeddings)

        sort_cand = [list_candidates[i] for i in range(len(labels)) if labels[i] == 0]
        sort_emb = [list_embeddings[i] for i in range(len(labels)) if labels[i] == 0]

        return sort_emb, sort_cand

    def personality(self, sort_emb_list, sort_cand_list):
        result = self.client.query('SELECT user_requests FROM users')
        user_requests = [row[0] for row in result.result_rows]
        
        cos_sim = {
            "word": [],
            "rank": []
        }

        for emb, word in zip(sort_emb_list, sort_cand_list):
            for user_request in user_requests:
                similarity = util.cos_sim(emb, user_request).cpu().numpy().tolist()[0][0]
                cos_sim["word"].append(word)
                cos_sim["rank"].append(similarity)
        
        df = pd.DataFrame(cos_sim)
        df_sorted = df.sort_values(by='rank', ascending=False)
        df_unique = df_sorted.drop_duplicates(subset='word', keep='first')
        
        return df_unique.to_dict(orient='list')

    def get_answer(self, request):
        try:
            sort_emb_list, sort_cand_list = self.clastering_req(request)
            if sort_emb_list and sort_cand_list:
                return self.personality(sort_emb_list, sort_cand_list)
            return request
        except IndexError:
            return request

In [46]:
service = AutocompleteService(client)

In [47]:
%%time
request = 'смег'
service.get_answer(request) # на CPU

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


CPU times: user 209 ms, sys: 3.59 ms, total: 213 ms
Wall time: 451 ms


{'word': ['смешные котики',
  'смешные приколы',
  'смешные коты',
  'смешные животные'],
 'rank': [1.0000001192092896, 1.0000001192092896, 1.0, 0.833310067653656]}